In [1]:
# Cargo todas las librerias necesarias para el modelo
import pandas as pd
import math
import time
import numpy as np
import numpy.matlib
import matplotlib.pyplot as plt
import mifs
from abc import ABCMeta, abstractmethod

C:\Users\Administrator\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:143: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
def create_dataset(dataset,look_back):
    k=np.shape(dataset)[0]
    dataset=dataset.reshape(1,k)
    datax=np.zeros((k-look_back-23,look_back))
    for i in range(0,look_back,1):
        datax[:,i]=dataset[:,i:(k-look_back+i-23)]
    dataY=np.zeros((k-look_back-23,24))
    for i in range(0,24,1):
        dataY[:,i]=dataset[:,(look_back+i):(k-23+i)]
    return datax,dataY
def Kerfun(X,Z,p1,p2):
    if X.shape[1]!= Z.shape[1]:
        K=[]
        display('The second dimensions for X and Z must be agree.')
        return K
    X1=np.tile((((X*X).sum(axis=1)).reshape(X.shape[0],1)),(1,Z.shape[0]))
    Z1=np.tile((((Z*Z).sum(axis=1)).reshape(1,Z.shape[0])),(X.shape[0],1))
    rbf=0.5*np.exp(np.dot(-p1,(X1+Z1-np.dot(np.dot(2,X),Z.T))))
    poly=0.4*((np.dot(X,Z.T) + p1) ** p2)
    line=0.1*np.dot(X,Z.T)
    K=rbf+poly+line
    return K
def MLSSVRTrain(trnX,trnY,gamma,p,p1):
    if (trnX.shape[0] != trnY.shape[0]):
        display('The number of rows in trnX and trnY must be equal.')
        return v,b    
    l,m=trnY.shape
    K=Kerfun(trnX,trnX,p,p1)
    D1=np.c_[K/gamma,np.zeros((l,1))]
    D=np.r_[D1,np.zeros((1,l+1))]
    G=np.c_[K,np.ones((l,1))]
    A=D+np.dot(G.T,G) #+np.eye(l+1)
    P=np.linalg.solve(A,np.dot(G.T,trnY))
    v=P[0:l]
    b = P[l]
    return b,v
def MLSSVRPredict(tstX,tstY,trnX,v,b,p,p1):   
    if (tstY.shape[1] != len(b)):
        display('The number of column in tstY and b must be equal.')
        return predictY,MSE
    m=tstY.shape[1]
    l=trnX.shape[0]
    if (v.shape[0] != l or v.shape[1]!= m):
        display('The size of alpha should be ' + l + '*' + m)
        return predictY,MSE
    tstN=tstX.shape[0]
    b=b.flatten('F')
    K=Kerfun(tstX,trnX,p,p1)
    predictY=np.dot(K,v) + np.tile(b.T,(tstN,1))
    MSE=max(pow((predictY-tstY),2).sum(axis=1))
    #MSE=np.mean(pow((predictY-tstY),2))
    return predictY,MSE

In [4]:
def func_transformer(func):
    prefered_function_format = '''
    def demo_func(x):
        x1, x2, x3 = x[:, 0], x[:, 1], x[:, 2]
        return x1 ** 2 + (x2 - 0.05) ** 2 + x3 ** 2
    '''
    is_vector = getattr(func, 'is_vector', False)
    if is_vector:
        return func
    else:
        if func.__code__.co_argcount == 1:
            def func_transformed(X):
                return np.array([func(x) for x in X])

            return func_transformed
        elif func.__code__.co_argcount > 1:

            def func_transformed(X):
                return np.array([func(*tuple(x)) for x in X])

            return func_transformed

    raise ValueError('''
    object function error,
    function should be like this:
    ''' + prefered_function_format)
class SkoBase(metaclass=ABCMeta):
    def register(self, operator_name, operator, *args, **kwargs):
        def operator_wapper(*wrapper_args):
            return operator(*(wrapper_args + args), **kwargs)
        setattr(self, operator_name, types.MethodType(operator_wapper, self))
        return self
class PSO1(SkoBase):
    def __init__(self, func, dim, pop=40, max_iter=150, lb=None, ub=None,T0=3,Tg=5,Tgz=10,w=0.8, c1=2, c2=2):
        self.func = func_transformer(func)
        self.wi = w  # inertia
        self.w = w
        self.we=0.3
        self.cp, self.cg = c1, c2  # parameters to control personal best, global best respectively
        self.pop = pop  # number of particles
        self.dim = dim  # dimension of particles, which is the number of variables of func
        self.max_iter = max_iter  # max iter
        self.T0=T0*np.ones((self.pop,1))
        self.Tg=Tg
        self.Tgz=Tgz
        self.has_constraints = not (lb is None and ub is None)
        self.lb = -np.ones(self.dim) if lb is None else np.array(lb)
        self.ub = np.ones(self.dim) if ub is None else np.array(ub)
        assert self.dim == len(self.lb) == len(self.ub), 'dim == len(lb) == len(ub) is not True'
        assert np.all(self.ub > self.lb), 'upper-bound must be greater than lower-bound'

        self.X = np.random.uniform(low=self.lb, high=self.ub, size=(self.pop, self.dim))
        self.X[:,self.dim-1]=np.round(self.X[:,self.dim-1])
        #print("self.X",self.X)
        self.Y = self.cal_y()  # y = f(x) for all particles
        self.pbest_x = self.X.copy()  # personal best location of every particle in history
        #print("self.pbest_x", self.pbest_x)
        self.pbest_y = self.Y.copy()  # best image of every particle in history
        self.gbest_x = np.zeros((1, self.dim))  # global best location for all particles
        self.gbest_y = np.inf  # global best y for all particles
        self.gbest_y_hist = []  # gbest_y of every iteration
        self.tg=0
        self.t0= np.zeros((self.pop,1))
        self.update_gbest()
        # record verbose values
        self.record_mode = False
        self.record_value = {'X': [],'Y': []}
    def update_X(self):
        #r3= np.where(self.t0 >self.T0, np.random.rand(self.pop, self.dim),1)
        r3= np.where(self.t0 >self.T0, \
            np.random.uniform(low=0,high=1,size=(self.pop, self.dim)),1)
        r4= np.where(self.tg >self.Tg,\
            np.random.uniform(low=0,high=1,size=(1,self.dim)),1)
        r1 = np.random.rand(self.pop, self.dim)
        r2 = np.random.rand(self.pop, self.dim)
        self.X=self.w *self.X+self.cp*r1*(r3*self.pbest_x-self.X)+\
               self.cg*r2*(r4*self.gbest_x - self.X)
        self.X[:,self.dim-1]=np.round(self.X[:,self.dim-1])
        if self.has_constraints:
            self.X = np.clip(self.X, self.lb, self.ub)
        #print("self.X",self.X) 
    def cal_y(self):
        # calculate y for every x in X
        self.Y = self.func(self.X).reshape(-1, 1)
        return self.Y

    def update_pbest(self):
        self.t0 = np.where(self.pbest_y > self.Y,0,self.t0+1)
        #print("self.t0",self.t0)
        self.pbest_x = np.where(self.pbest_y > self.Y, self.X, self.pbest_x)
        self.pbest_y = np.where(self.pbest_y > self.Y, self.Y, self.pbest_y)
        #print("self.pbest_x",self.pbest_x)
    def update_gbest(self):
        self.tg=self.tg+1
        if self.gbest_y > self.pbest_y.min():
            self.gbest_x = self.pbest_x[self.pbest_y.argmin(), :].copy()
            self.gbest_y = self.pbest_y.min()
            self.tg=0
        #print("self.gbest_x",self.gbest_x)
        #print("self.gbest_y",self.gbest_y)

    def recorder(self):
        if not self.record_mode:
            return
        self.record_value['X'].append(self.X)
        self.record_value['Y'].append(self.Y)

    def run(self, max_iter=None):
        self.max_iter = max_iter or self.max_iter
        for iter_num in range(self.max_iter):
            self.recorder()
            self.update_X()
            self.cal_y()
            self.update_pbest()
            self.update_gbest()
            self.gbest_y_hist.append(self.gbest_y)
            #self.w=self.wi-(self.wi-self.we)*(iter_num/self.max_iter)**2
        print("self.gbest_y",self.gbest_y)
        return self
    fit = run
def demo_func(gamma,p,p1):
    b,v=MLSSVRTrain(trainX,trainY,gamma,p,p1)
    tmpY,MSE=MLSSVRPredict(validX,validY,trainX,v,b,p,p1)
    #print(MSE)
    return MSE
#特征选择
def mrmry(trainx,trainy):
    x=trainx
    mrmr=[167]
    for i in range(0,6):
        y=trainy[:,i]
        feat_selector = mifs.MutualInformationFeatureSelector(method='MRMR',n_features=4,categorical=False)
        s=feat_selector.fit(x,y)
        mrmr=set(s.ranking_) | set(mrmr)
    return mrmr

In [9]:
imf1 = pd.read_csv("C:/Users/Administrator/Desktop/data/imf1.csv").values
imf2 = pd.read_csv("C:/Users/Administrator/Desktop/data/imf2.csv").values
imf3 = pd.read_csv("C:/Users/Administrator/Desktop/data/imf3.csv").values
imf4 = pd.read_csv("C:/Users/Administrator/Desktop/data/imf4.csv").values
imf5 = pd.read_csv("C:/Users/Administrator/Desktop/data/imf5.csv").values
imf6 = pd.read_csv("C:/Users/Administrator/Desktop/data/imf6.csv").values
look_back=168
trainximf1,trainyimf1=create_dataset(imf1[0:672],look_back)
validximf1,validyimf1=create_dataset(imf1[480:912],look_back)
testximf1,testyimf1=create_dataset(imf1[720:1009],look_back)#12月1号
trainximf2,trainyimf2=create_dataset(imf2[0:672],look_back)
validximf2,validyimf2=create_dataset(imf2[480:912],look_back)
testximf2,testyimf2=create_dataset(imf2[720:1009],look_back)#12月1号

trainximf3,trainyimf3=create_dataset(imf3[0:672],look_back)
validximf3,validyimf3=create_dataset(imf3[480:912],look_back)
testximf3,testyimf3=create_dataset(imf3[720:1009],look_back)#12月1号

trainximf4,trainyimf4=create_dataset(imf4[0:672],look_back)
validximf4,validyimf4=create_dataset(imf4[480:912],look_back)
testximf4,testyimf4=create_dataset(imf4[720:1009],look_back)#12月1号

trainximf5,trainyimf5=create_dataset(imf5[0:672],look_back)
validximf5,validyimf5=create_dataset(imf5[480:912],look_back)
testximf5,testyimf5=create_dataset(imf5[720:1009],look_back)#12月1号

trainximf6,trainyimf6=create_dataset(imf6[0:672],look_back)
validximf6,validyimf6=create_dataset(imf6[480:912],look_back)
testximf6,testyimf6=create_dataset(imf6[720:1009],look_back)#12月1号
for j in range(0,9,1):
    s=np.shape(testyimf6)[0]
    predy=np.zeros((s,0))
    for i in range(4):
        index1=i*6
        index2=index1+6
        trainY=trainyimf1[:,index1:index2]
        validY=validyimf1[:,index1:index2]
        testY=testyimf1[:,index1:index2]
        #特征选择
        mrmr=mrmry(trainximf1,trainY)
        trainX=trainximf1[:,list(mrmr)]
        validX=validximf1[:,list(mrmr)]
        testX=testximf1[:,list(mrmr)]
        #参数选择
        pso=PSO1(func=demo_func,dim=3,pop=20,max_iter=100,lb=[0.001, 0.001, 1],ub=[1000, 1000,10],w=0.8,c1=2,c2=2)
        pso.run()
        b,alpha=MLSSVRTrain(trainX,trainY,pso.gbest_x[0],pso.gbest_x[1],pso.gbest_x[2])
        predictY,Mst=MLSSVRPredict(testX,testY,trainX,alpha,b,pso.gbest_x[1],pso.gbest_x[2])
        predy=np.c_[predy,predictY]
    dt = pd.DataFrame(predy)
    url="C:/Users/Administrator/Desktop/data/电力负荷预测/t检验/IMF1zMIMO"+str(j)+".csv"
    dt.to_csv(url,index=0)
    predy=np.zeros((s,0))
    for i in range(4):
        index1=i*6
        index2=index1+6
        trainY=trainyimf2[:,index1:index2]
        validY=validyimf2[:,index1:index2]
        testY=testyimf2[:,index1:index2]
        #特征选择
        mrmr=mrmry(trainximf2,trainY)
        trainX=trainximf2[:,list(mrmr)]
        validX=validximf2[:,list(mrmr)]
        testX=testximf2[:,list(mrmr)]
        #参数选择
        pso=PSO1(func=demo_func,dim=3,pop=20,max_iter=100,lb=[0.001, 0.001, 1],ub=[1000, 1000,10],w=0.8,c1=2,c2=2)
        pso.run()
        b,alpha=MLSSVRTrain(trainX,trainY,pso.gbest_x[0],pso.gbest_x[1],pso.gbest_x[2])
        predictY,Mst=MLSSVRPredict(testX,testY,trainX,alpha,b,pso.gbest_x[1],pso.gbest_x[2])
        predy=np.c_[predy,predictY]
    dt = pd.DataFrame(predy)
    url="C:/Users/Administrator/Desktop/data/电力负荷预测/t检验/IMF2zMIMO"+str(j)+".csv"
    dt.to_csv(url,index=0)
    predy=np.zeros((s,0))
    for i in range(4):
        index1=i*6
        index2=index1+6
        trainY=trainyimf3[:,index1:index2]
        validY=validyimf3[:,index1:index2]
        testY=testyimf3[:,index1:index2]
        #特征选择
        mrmr=mrmry(trainximf3,trainY)
        trainX=trainximf3[:,list(mrmr)]
        validX=validximf3[:,list(mrmr)]
        testX=testximf3[:,list(mrmr)]
        #参数选择
        pso=PSO1(func=demo_func,dim=3,pop=20,max_iter=100,lb=[0.001, 0.001, 1],ub=[1000, 1000,10],w=0.8,c1=2,c2=2)
        pso.run()
        b,alpha=MLSSVRTrain(trainX,trainY,pso.gbest_x[0],pso.gbest_x[1],pso.gbest_x[2])
        predictY,Mst=MLSSVRPredict(testX,testY,trainX,alpha,b,pso.gbest_x[1],pso.gbest_x[2])
        predy=np.c_[predy,predictY]
    dt = pd.DataFrame(predy)
    url="C:/Users/Administrator/Desktop/data/电力负荷预测/t检验/IMF3zMIMO"+str(j)+".csv"
    dt.to_csv(url,index=0)
    predy=np.zeros((s,0))
    for i in range(4):
        index1=i*6
        index2=index1+6
        trainY=trainyimf4[:,index1:index2]
        validY=validyimf4[:,index1:index2]
        testY=testyimf4[:,index1:index2]
        #特征选择
        mrmr=mrmry(trainximf4,trainY)
        trainX=trainximf4[:,list(mrmr)]
        validX=validximf4[:,list(mrmr)]
        testX=testximf4[:,list(mrmr)]
        #参数选择
        pso=PSO1(func=demo_func,dim=3,pop=20,max_iter=100,lb=[0.001, 0.001, 1],ub=[1000, 1000,10],w=0.8,c1=2,c2=2)
        pso.run()
        b,alpha=MLSSVRTrain(trainX,trainY,pso.gbest_x[0],pso.gbest_x[1],pso.gbest_x[2])
        predictY,Mst=MLSSVRPredict(testX,testY,trainX,alpha,b,pso.gbest_x[1],pso.gbest_x[2])
        predy=np.c_[predy,predictY]
    dt = pd.DataFrame(predy)
    url="C:/Users/Administrator/Desktop/data/电力负荷预测/t检验/IMF4zMIMO"+str(j)+".csv"
    dt.to_csv(url,index=0)
    predy=np.zeros((s,0))
    for i in range(4):
        index1=i*6
        index2=index1+6
        trainY=trainyimf5[:,index1:index2]
        validY=validyimf5[:,index1:index2]
        testY=testyimf5[:,index1:index2]
        #特征选择
        mrmr=mrmry(trainximf5,trainY)
        trainX=trainximf5[:,list(mrmr)]
        validX=validximf5[:,list(mrmr)]
        testX=testximf5[:,list(mrmr)]
        #参数选择
        pso=PSO1(func=demo_func,dim=3,pop=20,max_iter=100,lb=[0.001, 0.001, 1],ub=[1000, 1000,10],w=0.8,c1=2,c2=2)
        pso.run()
        b,alpha=MLSSVRTrain(trainX,trainY,pso.gbest_x[0],pso.gbest_x[1],pso.gbest_x[2])
        predictY,Mst=MLSSVRPredict(testX,testY,trainX,alpha,b,pso.gbest_x[1],pso.gbest_x[2])
        predy=np.c_[predy,predictY]
    dt = pd.DataFrame(predy)
    url="C:/Users/Administrator/Desktop/data/电力负荷预测/t检验/IMF5zMIMO"+str(j)+".csv"
    dt.to_csv(url,index=0)
    predy=np.zeros((s,0))
    for i in range(4):
        index1=i*6
        index2=index1+6
        trainY=trainyimf6[:,index1:index2]
        validY=validyimf6[:,index1:index2]
        testY=testyimf6[:,index1:index2]
        #特征选择
        mrmr=mrmry(trainximf6,trainY)
        trainX=trainximf6[:,list(mrmr)]
        validX=validximf6[:,list(mrmr)]
        testX=testximf6[:,list(mrmr)]
        #参数选择
        pso=PSO1(func=demo_func,dim=3,pop=20,max_iter=100,lb=[0.001, 0.001, 1],ub=[1000, 1000,10],w=0.8,c1=2,c2=2)
        pso.run()
        b,alpha=MLSSVRTrain(trainX,trainY,pso.gbest_x[0],pso.gbest_x[1],pso.gbest_x[2])
        predictY,Mst=MLSSVRPredict(testX,testY,trainX,alpha,b,pso.gbest_x[1],pso.gbest_x[2])
        predy=np.c_[predy,predictY]
    dt = pd.DataFrame(predy)
    url="C:/Users/Administrator/Desktop/data/电力负荷预测/t检验/IMF6zMIMO"+str(j)+".csv"
    dt.to_csv(url,index=0)
    print(j)

self.gbest_y 0.001324251741254457
self.gbest_y 0.02666669896982801
self.gbest_y 0.042106339037125635
self.gbest_y 0.1198292545759569
self.gbest_y 0.010119730149331124
self.gbest_y 0.03597736078198819
self.gbest_y 0.02497651423608862
self.gbest_y 0.027518957606673063
self.gbest_y 0.001808997679151307
self.gbest_y 0.012591634263681302
self.gbest_y 0.013706598707578916
self.gbest_y 0.013860646621939853
self.gbest_y 0.0012442662006248747
self.gbest_y 0.004498292545700248
self.gbest_y 0.0038526681619240437
self.gbest_y 0.005036739468663305
self.gbest_y 0.0005079155054294888
self.gbest_y 0.0021638562098632587
self.gbest_y 0.0023444597965515948
self.gbest_y 0.002256688331078867
self.gbest_y 0.001504454497877763
self.gbest_y 0.001903578229001164
self.gbest_y 0.0019081638713605807
self.gbest_y 0.0019262322608908434
0
self.gbest_y 0.0011615532194745705
self.gbest_y 0.021821631058970902
self.gbest_y 0.04190434800045581
self.gbest_y 0.12126532109868438
self.gbest_y 0.010734124668535516
self.gbest_